In [ ]:
import matplotlib.pyplot as plt
images= ['barn_mountains','logo','peppers']
qualities = [75,50,25]
from PIL import Image
for s in images:
    img = Image.open(f"imagens/{s}.bmp")
    rgb_img = img.convert('RGB')
    for q in qualities:
        rgb_img.save(f"ex1/{s}_{q}.jpeg", quality=q)
        im2 = Image.open(f"ex1/{s}_{q}.jpeg")
        plt.figure()
        plt.title(f'{s}_{q}.jpeg')
        plt.imshow(im2)


# Results


In [ ]:
def encoder():
    pass

def decoder():
    pass